In [135]:
from tqdm.notebook import tqdm
import pandas as pd
from datasets import Dataset
from src.dataset import SPFastaDatasetBinary, SPFastaDatasetBinaryWithTokenizedCategory
import torch
from torch import nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AdamW
from torch.utils.data import DataLoader
import numpy as np
import os
import matplotlib.pyplot as plt
from transformers import AutoModelForSequenceClassification
from os import path


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [136]:
ds_type = SPFastaDatasetBinaryWithTokenizedCategory
def load_data(path: path):
    ds = ds_type(path)
    return Dataset.from_pandas(ds.data).with_format("torch", device=device)
dataset_train = load_data("data/train.fasta")
dataset_val, dataset_test = load_data("./data/test.fasta").train_test_split(0.8).values()

100%|██████████| 8811/8811 [00:00<00:00, 980109.60it/s]


In [137]:
# def load_data_with_size_uniform(path, size=1000):
#     ds = SPFastaDatasetBinaryWithTokenizedCategory(path)
#     neg_labels = ds.data.loc[ds.data['labels'] == 0][:size//2]
#     pos_labels = ds.data.loc[ds.data['labels'] == 1][:size//2]
#     combi = pd.concat((neg_labels, pos_labels))
#     return Dataset.from_pandas(combi).with_format("torch", device=device)

# dataset_train = load_data_with_size_uniform("data/train.fasta", 100)
# dataset_test  = load_data_with_size_uniform("data/test.fasta", 100)

In [138]:
from datasets import ClassLabel
dataset_train = dataset_train.cast_column('labels', ClassLabel(num_classes = 2, names=["NO_SP", "SP"]))
dataset_val = dataset_val.cast_column('labels', ClassLabel(num_classes = 2, names=["NO_SP", "SP"]))
dataset_test = dataset_test.cast_column('labels', ClassLabel(num_classes = 2, names=["NO_SP", "SP"]))
dataset_train[0]['labels']

Casting the dataset:   0%|          | 0/20290 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1762 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/7049 [00:00<?, ? examples/s]

tensor(0, device='cuda:0')

In [139]:
# Tokenize:
tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_bert", device="cpu")
dataset_train = dataset_train.map(lambda x: tokenizer(x['text'], return_tensors="pt", padding='max_length', max_length=81, truncation=True), batched=True)
dataset_val = dataset_val.map(lambda x: tokenizer(x['text'], return_tensors="pt", padding='max_length', max_length=81, truncation=True), batched=True)
dataset_test = dataset_test.map(lambda x: tokenizer(x['text'], return_tensors="pt", padding='max_length', max_length=81, truncation=True), batched=True)
dataset_train, dataset_val, dataset_test

Map:   0%|          | 0/20290 [00:00<?, ? examples/s]

Map:   0%|          | 0/1762 [00:00<?, ? examples/s]

Map:   0%|          | 0/7049 [00:00<?, ? examples/s]

(Dataset({
     features: ['text', 'labels', 'uniprot_ac', 'kingdom', 'type', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 20290
 }),
 Dataset({
     features: ['text', 'labels', 'uniprot_ac', 'kingdom', 'type', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 1762
 }),
 Dataset({
     features: ['text', 'labels', 'uniprot_ac', 'kingdom', 'type', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 7049
 }))

In [140]:
import evaluate
acc = evaluate.load("accuracy")
pre = evaluate.load("precision")
rec = evaluate.load("recall")
f1m = evaluate.load("f1")

metrics = [acc, pre, rec, f1m]

def compute_metrics(eval_pred):
    logits, y = eval_pred
    x = logits.argmax(-1)
    return {k: v for metric in [m.compute(predictions=x, references=y) for m in metrics] for k, v in metric.items()}

# Define model

In [141]:
from transformers import AutoModelForSequenceClassification


model = AutoModelForSequenceClassification.from_pretrained(
    #"Rostlab/prot_bert",
    "microsoft/MiniLM-L12-H384-uncased",
    num_labels=2,
    label2id = {
        'NO_SP': 0,
        'SP': 1
    },
    id2label= {
        0: 'NO_SP',
        1: 'SP'
}).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Define FocalLoss, compute class weights and create a custom Trainer

In [142]:
class FocalLoss(nn.Module):
    def __init__(self, weights, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = weights
        self.gamma = gamma

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        loss = (self.alpha[targets] * (1 - pt) ** self.gamma * ce_loss).mean()
        return loss

In [143]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.array([0, 1]), y=dataset_train['labels'].cpu().numpy())
class_weights = torch.tensor(class_weights).to(device).float()
class_weights

tensor([0.6493, 2.1747], device='cuda:0')

In [144]:
from transformers import Trainer

class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        
        outputs = model(**inputs)
        logits = outputs.get('logits')
        labels = inputs.get('labels')

        loss_fn = FocalLoss(weights=class_weights)
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [145]:
from transformers import TrainingArguments

# batch_size = 8
epochs = 50
logging_steps=1000
eval_steps=10000

args = TrainingArguments(
    output_dir='./model',
    logging_dir="./logs",
    num_train_epochs=epochs,
    learning_rate=2e-5,
    logging_first_step=True,
    weight_decay=0.0001,
    evaluation_strategy='steps',
    save_strategy='epoch',
    logging_steps=logging_steps,
    eval_steps=eval_steps,
    fp16=True,
    optim='adamw_torch',
    remove_unused_columns=True,
    auto_find_batch_size=True,
)

In [146]:
trainer = WeightedLossTrainer(
    model=model,
    args=args,
    compute_metrics=compute_metrics,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    tokenizer=tokenizer
)
trainer.train()

  0%|          | 0/126850 [00:00<?, ?it/s]

{'loss': 0.2456, 'learning_rate': 1.9999842333464724e-05, 'epoch': 0.0}
{'loss': 0.1742, 'learning_rate': 1.9842333464722114e-05, 'epoch': 0.39}
{'loss': 0.1345, 'learning_rate': 1.9685297595585336e-05, 'epoch': 0.79}
{'loss': 0.0825, 'learning_rate': 1.952763106030745e-05, 'epoch': 1.18}
{'loss': 0.0691, 'learning_rate': 1.9370122191564842e-05, 'epoch': 1.58}
{'loss': 0.0593, 'learning_rate': 1.9212455656286955e-05, 'epoch': 1.97}
{'loss': 0.0559, 'learning_rate': 1.9054789121009067e-05, 'epoch': 2.36}
{'loss': 0.0505, 'learning_rate': 1.889712258573118e-05, 'epoch': 2.76}
{'loss': 0.0585, 'learning_rate': 1.8739456050453293e-05, 'epoch': 3.15}
{'loss': 0.0496, 'learning_rate': 1.8581789515175405e-05, 'epoch': 3.55}
{'loss': 0.0504, 'learning_rate': 1.84242806464328e-05, 'epoch': 3.94}


  0%|          | 0/221 [00:00<?, ?it/s]

{'eval_loss': 0.02506309188902378, 'eval_accuracy': 0.9750283768444948, 'eval_precision': 0.8692307692307693, 'eval_recall': 0.9576271186440678, 'eval_f1': 0.9112903225806451, 'eval_runtime': 6.3761, 'eval_samples_per_second': 276.344, 'eval_steps_per_second': 34.661, 'epoch': 3.94}
{'loss': 0.0442, 'learning_rate': 1.8266614111154908e-05, 'epoch': 4.34}
{'loss': 0.0467, 'learning_rate': 1.8109105242412298e-05, 'epoch': 4.73}
{'loss': 0.0448, 'learning_rate': 1.795159637366969e-05, 'epoch': 5.12}
{'loss': 0.0418, 'learning_rate': 1.7793929838391804e-05, 'epoch': 5.52}
{'loss': 0.0339, 'learning_rate': 1.7636263303113916e-05, 'epoch': 5.91}
{'loss': 0.0385, 'learning_rate': 1.747859676783603e-05, 'epoch': 6.31}
{'loss': 0.0425, 'learning_rate': 1.732093023255814e-05, 'epoch': 6.7}
{'loss': 0.0413, 'learning_rate': 1.716342136381553e-05, 'epoch': 7.09}
{'loss': 0.039, 'learning_rate': 1.7005754828537644e-05, 'epoch': 7.49}
{'loss': 0.0398, 'learning_rate': 1.6848088293259757e-05, 'epoch'

  0%|          | 0/221 [00:00<?, ?it/s]

...

  0%|          | 0/221 [00:00<?, ?it/s]

{'eval_loss': 0.018768297508358955, 'eval_accuracy': 0.9954597048808173, 'eval_precision': 0.9956521739130435, 'eval_recall': 0.9703389830508474, 'eval_f1': 0.9828326180257511, 'eval_runtime': 5.9996, 'eval_samples_per_second': 293.685, 'eval_steps_per_second': 36.836, 'epoch': 43.36}
{'loss': 0.0086, 'learning_rate': 2.5059519117067405e-06, 'epoch': 43.75}
{'loss': 0.0092, 'learning_rate': 2.348285376428853e-06, 'epoch': 44.15}
{'loss': 0.0099, 'learning_rate': 2.190618841150966e-06, 'epoch': 44.54}
{'loss': 0.0101, 'learning_rate': 2.0331099724083565e-06, 'epoch': 44.93}
{'loss': 0.0079, 'learning_rate': 1.8754434371304691e-06, 'epoch': 45.33}
{'loss': 0.0102, 'learning_rate': 1.71793456838786e-06, 'epoch': 45.72}
{'loss': 0.0086, 'learning_rate': 1.5602680331099725e-06, 'epoch': 46.12}
{'loss': 0.0097, 'learning_rate': 1.4026014978320853e-06, 'epoch': 46.51}
{'loss': 0.0048, 'learning_rate': 1.2449349625541981e-06, 'epoch': 46.91}
{'loss': 0.0112, 'learning_rate': 1.0872684272763107

  0%|          | 0/221 [00:00<?, ?it/s]

{'eval_loss': 0.008344221860170364, 'eval_accuracy': 0.996594778660613, 'eval_precision': 0.9872881355932204, 'eval_recall': 0.9872881355932204, 'eval_f1': 0.9872881355932204, 'eval_runtime': 6.0461, 'eval_samples_per_second': 291.429, 'eval_steps_per_second': 36.553, 'epoch': 47.3}
{'loss': 0.0069, 'learning_rate': 9.297595585337013e-07, 'epoch': 47.69}
{'loss': 0.0103, 'learning_rate': 7.72250689791092e-07, 'epoch': 48.09}
{'loss': 0.0079, 'learning_rate': 6.145841545132046e-07, 'epoch': 48.48}
{'loss': 0.0061, 'learning_rate': 4.5691761923531733e-07, 'epoch': 48.88}
{'loss': 0.0081, 'learning_rate': 2.994087504927079e-07, 'epoch': 49.27}
{'loss': 0.0101, 'learning_rate': 1.4174221521482067e-07, 'epoch': 49.66}
{'train_runtime': 11129.2553, 'train_samples_per_second': 91.156, 'train_steps_per_second': 11.398, 'train_loss': 0.024358350507001685, 'epoch': 50.0}


TrainOutput(global_step=126850, training_loss=0.024358350507001685, metrics={'train_runtime': 11129.2553, 'train_samples_per_second': 91.156, 'train_steps_per_second': 11.398, 'train_loss': 0.024358350507001685, 'epoch': 50.0})

In [152]:
model.save_pretrained("./model/checkpoint-final")
tokenizer.save_pretrained("./model/checkpoint-final")

('./model/checkpoint-final\\tokenizer_config.json',
 './model/checkpoint-final\\special_tokens_map.json',
 './model/checkpoint-final\\vocab.txt',
 './model/checkpoint-final\\added_tokens.json',
 './model/checkpoint-final\\tokenizer.json')

In [153]:
model = AutoModelForSequenceClassification.from_pretrained("./model/checkpoint-final").to(device)
tokenizer = AutoTokenizer.from_pretrained("./model/checkpoint-final")

In [154]:
samples = dataset_test.select_columns(['input_ids', 'attention_mask', 'labels'])
sample = samples[torch.randint(len(samples), (1,))]
with torch.no_grad():
    pred = model(**sample).logits.argmax(-1)
pred-sample['labels']

tensor([0], device='cuda:0')

In [155]:
res = []
for batch in tqdm(torch.utils.data.DataLoader(samples, batch_size=8)):
    pred = model(**batch).logits.argmax(-1)
    for p, l in zip(pred, batch['labels']):
        res.append((p.item(), l.item()))

  0%|          | 0/882 [00:00<?, ?it/s]

In [156]:
pd.DataFrame(res, columns=['pred', 'label']).value_counts()

pred  label
0     0        6039
1     1         983
      0          15
0     1          12
Name: count, dtype: int64

```python
# TODO: check if this is correct
loss_weights = torch.tensor([neg_weight if x[0] == 1 else pos_weight for x in batch['label']])
loss_weights = torch.tensor([neg_weight, pos_weight]).to(device) # or this
model_v = "microsoft/MiniLM-L12-H384-uncased"

# Classical training loop:
import pickle
loss_log = []
epoch0 = 0
if os.path.exists("model/temp.state"):
    with open("model/temp.state", "rb") as f:
        state = pickle.load(f)
        loss_log = state['loss_log']
        epoch0 = state['epoch']

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
epochs = 10
batch_size = 4

model.train()
batches = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
for epoch in range(epoch0, epochs):
    for i, batch in enumerate(tqdm(batches)):
        class_weights = torch.tensor([neg_weight if x[0] == 1 else pos_weight for x in batch['labels']]).to(device)

        optimizer.zero_grad()
        loss = model.loss(batch, weights=class_weights)

        loss.backward()
        optimizer.step()
        loss_log.append(loss.cpu().detach().item())
        if (i+1) % (len(batches) // 20) == 0:
            print(np.mean(loss_log[-5000:]))
```